In [9]:
import numpy as np
import cv2 as cv

In [10]:
cap = cv.VideoCapture('../src/video.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

# Parameters for lucas kanade optical flow
lk_params = dict( winSize = (15, 15), maxLevel = 2, criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_frame = cv.resize(old_frame, (0,0), fx=0.5, fy=0.5)
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, frame = cap.read()

    if not ret:
        print('No frames grabbed!')
        break

    frame = cv.resize(frame, (0,0), fx=0.5, fy=0.5)
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

    img = cv.add(frame, mask)
    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff

    if k == 27:
        break
    
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv.destroyAllWindows()

# Questão 1

In [4]:
# Inicializa a captura de vídeo
cap = cv.VideoCapture('../src/video.mp4')

# Inicializa o detector ORB com o número máximo de pontos-chave (100)
orb = cv.ORB_create(nfeatures=100)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Cria algumas cores aleatórias
color = np.random.randint(0, 255, (100, 3))

# Lê o primeiro quadro e encontra pontos-chave nele
ret, old_frame = cap.read()
old_frame = cv.resize(old_frame, (0, 0), fx=0.5, fy=0.5)
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

# Detecta os pontos-chave usando o ORB
kp0 = orb.detect(old_gray, None)
kp0 = sorted(kp0, key=lambda x: x.response, reverse=True)[:100]  # Limita a 100 pontos-chave
p0 = np.array([kp.pt for kp in kp0], dtype=np.float32).reshape(-1, 1, 2)

# Cria uma máscara para desenhar
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    # Redimensiona e converte para escala de cinza
    frame = cv.resize(frame, (0, 0), fx=0.5, fy=0.5)
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calcula o fluxo óptico
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Seleciona os bons pontos
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

    # Desenha as trilhas
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

    # Combina a imagem com a máscara
    img = cv.add(frame, mask)

    # Exibe o quadro
    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff

    # Pressione ESC para sair
    if k == 27:
        break

    # Atualiza o quadro anterior e os pontos-chave
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv.destroyAllWindows()

No frames grabbed!


# Questão 2

In [5]:
# Inicializa a captura de vídeo
cap = cv.VideoCapture('../src/video2.mp4')

# Inicializa o detector ORB com o número máximo de pontos-chave (100)
orb = cv.ORB_create(nfeatures=100)

# Parâmetros para o fluxo óptico de Lucas-Kanade
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Cria algumas cores aleatórias
color = np.random.randint(0, 255, (100, 3))

# Define um limiar para o erro de estimativa
error_threshold = 0.5

# Lê o primeiro quadro e encontra pontos-chave nele
ret, old_frame = cap.read()
old_frame = cv.resize(old_frame, (0, 0), fx=0.5, fy=0.5)
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

# Detecta os pontos-chave usando o ORB
kp0 = orb.detect(old_gray, None)
kp0 = sorted(kp0, key=lambda x: x.response, reverse=True)[:100]  # Limita a 100 pontos-chave
p0 = np.array([kp.pt for kp in kp0], dtype=np.float32).reshape(-1, 1, 2)

# Cria uma máscara para desenhar
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    # Redimensiona e converte para escala de cinza
    frame = cv.resize(frame, (0, 0), fx=0.5, fy=0.5)
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calcula o fluxo óptico
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Seleciona os bons pontos com base no erro de estimativa
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        err_good = err[st == 1]

    # Desenha as trilhas apenas para os pontos cujo erro esteja abaixo do limiar
    for i, (new, old, e) in enumerate(zip(good_new, good_old, err_good)):
        if e < error_threshold:  # Verifica se o erro é menor que o limiar
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

    # Combina a imagem com a máscara
    img = cv.add(frame, mask)

    # Exibe o quadro
    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff

    # Pressione ESC para sair
    if k == 27:
        break

    # Atualiza o quadro anterior e os pontos-chave
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv.destroyAllWindows()

No frames grabbed!


# Questão 3

In [11]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(cv.samples.findFile('../src/video2.mp4'))
ret, frame1 = cap.read()
frame1 = cv.resize(frame1, (0, 0), fx=0.5, fy=0.5)

pyr_scale = 0.5
levels = 3
winsize = 15
iterations = 3
poly_n = 5
poly_sigma = 1.2
flags = 0

prvs_frame = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

# Taking a matrix of size 5 as the kernel
kernel = np.ones((9, 9), np.uint8)

while(1):
    ret, frame2 = cap.read()

    if not ret:
        print('No frames grabbed!')
        break

    frame2 = cv.resize(frame2, (0, 0), fx=0.5, fy=0.5)
    
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs_frame, next, None, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])

    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)

    if cv.waitKey(1) == ord('q'):
        break
    prvs_frame = next  # Corrigido aqui

cv.destroyAllWindows()


In [12]:
def mov_filter(video):
        cap = cv.VideoCapture(cv.samples.findFile(video))
        ret, frame1 = cap.read()

        # Verifica se o primeiro frame foi lido corretamente
        if not ret:
            print('Erro ao carregar o vídeo.')
            cap.release()
            cv.destroyAllWindows()
        else:
            frame1 = cv.resize(frame1, (0, 0), fx=0.5, fy=0.5)

        pyr_scale = 0.5
        levels = 3
        winsize = 15
        iterations = 3
        poly_n = 5
        poly_sigma = 1.2
        flags = 0

        prvs_frame = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

        # Kernel para reduzir ruído
        kernel = np.ones((5, 5), np.uint8)

        while True:
            ret, frame2 = cap.read()

            # Verifica se o frame foi lido corretamente
            if not ret:
                print('No frames grabbed!')
                break

            frame2 = cv.resize(frame2, (0, 0), fx=0.5, fy=0.5)

            # Conversão para escala de cinza
            next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
            
            # Calcula o fluxo óptico
            flow = cv.calcOpticalFlowFarneback(prvs_frame, next, None, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, 0)
            mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])

            # Normaliza a magnitude do fluxo para a faixa [0, 255]
            mag_norm = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)

            # Aplica um limite (threshold) para detectar movimento significativo
            _, mask = cv.threshold(mag_norm, 15, 255, cv.THRESH_BINARY)
            
            # Reduz o ruído utilizando uma operação morfológica de abertura
            mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
            
            # Certifica que a máscara é do mesmo tipo de dados que o frame (uint8)
            mask = mask.astype(np.uint8)
            
            # Converte a máscara para 3 canais para aplicá-la à imagem colorida
            mask_3c = cv.cvtColor(mask, cv.COLOR_GRAY2BGR)
            
            # Exibe os pixels em movimento com a cor original, e o resto preto
            moving_frame = cv.bitwise_and(frame2, mask_3c)

            # Mostra o resultado
            cv.imshow('Movimento', moving_frame)

            # Pressione 'q' para sair
            if cv.waitKey(1) == ord('q'):
                break

            # Atualiza o frame anterior
            prvs_frame = next

        cap.release()
        cv.destroyAllWindows()


In [13]:
mov_filter('../src/video2.mp4')

# Questão 4

In [10]:
mov_filter('../src/video.mp4')

O resultado observado nesse vídeo dos cachorros é bem pior do que no vídeo pois nesse caso não só os cachorros estão em movimento como a câmera também, ou seja, todos os pixels acabam ficando em movimento, não sendo filtrado somente os cachorros correndo. Diferente do caso do vídeo do homem carregando a caixa, em que a câmera esta parada, sendo exibido somente o movimento do homem e do cronômetro.